In [6]:
import json
import pandas as pd
from bs4 import BeautifulSoup

In [7]:
# Chemins vers les fichiers JSON d'entraînement et de développement
train_path = 'SCOTUS/train.json'
dev_path = 'SCOTUS/dev.json'

In [8]:
# Chargement des données d'entraînement à partir du fichier JSON
with open(train_path, 'r', encoding="utf-8") as f:
    train = json.load(f)

In [9]:
def html_processing(html_content):
    """
    Récupère le contenu intéressant du HTML en conservant les balises <a>, <em> et <blockquote>.
    
    Parameters:
    html_content (str): Le contenu HTML à traiter.

    Returns:
    list: Une liste de chaînes contenant le texte traité et les balises spécifiées.
    """
    soup = BeautifulSoup(html_content, 'html.parser')

    # On suppose que le contenu intéressant se trouve dans cette classe
    div_class = "-display-inline-block text-left"
    target_div = soup.find('div', class_=div_class)

    temp_result = []  # Liste pour stocker le texte traité
    if target_div:
        to_merge = False  # Indicateur pour savoir si on doit fusionner les textes
        for element in target_div.children:
            # Ignorer les sauts de ligne
            if element.getText() == '\n':
                continue
            # Traiter les balises <a> et <em>
            elif element.name in ["a", "em"]:
                temp_result.append(f"<{element.name}> {element.get_text()} </{element.name}>")
                to_merge = True
            # Si l'élément est du texte à fusionner
            elif to_merge:
                if temp_result[-1].startswith("<em>"):
                    temp_result[-2] += temp_result[-1] + element.get_text()
                else:
                    temp_result[-2] += " " + temp_result[-1] + " " + element.get_text()
                temp_result.pop()  # Retirer le dernier élément
                to_merge = False
            # Traiter les balises <blockquote>
            elif element.name == "blockquote":
                temp_result.append(f"<{element.name}>{element.get_text()}</{element.name}>")
            # Pour le texte normal
            else:
                temp_result.append(element.get_text())
    else:
        print(f"Aucun div trouvé avec la classe '{div_class}'.")

    result = []  # Liste finale pour le résultat

    # Nettoyer et fusionner le texte dans le résultat final
    for res in temp_result:
        # Remplacer les tabulations et les sauts de ligne
        if '\t' in res:
            string = res.replace('\t', '').replace('\n', ' ')
            result.append(string)
        else:
            if len(result) < 1:
                result.append(res.replace('\n', ''))
            else:
                result[-1] += " " + res.replace('\n', '')

    return result

In [10]:
# Traitement des données d'entraînement
for i in range(len(train)):
    html_content = train[i]["raw_source"]  # Récupérer le contenu HTML

    text = html_processing(html_content)  # Traiter le contenu HTML

    name = f'data_txt_save/train_{i}.txt'  # Nom du fichier de sortie

    # Écrire le texte traité dans un fichier
    with open(name, 'w', encoding='utf-8') as f:
        for line in text:
            f.write(line + '\n')